# Training Tests for Part Of Speech tagging

This notebook is dedicated to start working with the PoS dataset already pre-processed and the column networks that I'm creating.

The network will be constructed from small parts, each will be trained on top of the previous one, adding a new column and decoder.


In [1]:
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from langmodels.models import *
import langmodels.utf8codec as utf8codec
from langmodels.utils.tools import *
from langmodels.utils.helpers import *
from langmodels.utils.preprocess_conllu import *
from langmodels.train import *

/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/leo/venv3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dty

In [2]:
fcodebook

'/home/leo/projects/minibrain/predictors/sequence/text/utf8-codes/utf8_codebook_overfit_matrix_2seg_dim64.npy'

In [3]:
utf8codes = np.load(fcodebook)
utf8codes = utf8codes.reshape(1987, 64)

In [4]:
model = GatedConv1DPoS(utf8codes)

In [5]:
# load pretrained from saved Conv1D model
model.network.old_load_model("./trained_models/conv1dcol/","conv1dcol_nll-loss_epoch-79_001")

In [6]:
%%time
path = "./trained_models/GatedConv1DCol"
base_name = "GatedConv1DPoS_nll-loss"
train_test(model, path, base_name, max_data=45)

====> Timestamp 2019-11-29 16:14:19.379404 Epoch: 1 Average loss: 0.01313078
====> Timestamp 2019-11-29 16:16:10.513596 Epoch: 2 Average loss: 0.00784747
====> Timestamp 2019-11-29 16:18:01.780184 Epoch: 3 Average loss: 0.00737822
====> Timestamp 2019-11-29 16:19:53.327939 Epoch: 4 Average loss: 0.00681860
====> Timestamp 2019-11-29 16:21:44.910042 Epoch: 5 Average loss: 0.00653033
====> Timestamp 2019-11-29 16:23:36.287940 Epoch: 6 Average loss: 0.00623974
====> Timestamp 2019-11-29 16:25:27.709583 Epoch: 7 Average loss: 0.00601973
====> Timestamp 2019-11-29 16:27:19.147079 Epoch: 8 Average loss: 0.00554546
====> Timestamp 2019-11-29 16:29:09.132243 Epoch: 9 Average loss: 0.00530290
====> Timestamp 2019-11-29 16:31:01.551584 Epoch: 10 Average loss: 0.00532029
====> Timestamp 2019-11-29 16:32:55.329889 Epoch: 11 Average loss: 0.00521510
====> Timestamp 2019-11-29 16:34:49.958010 Epoch: 12 Average loss: 0.00502656
====> Timestamp 2019-11-29 16:36:44.820244 Epoch: 13 Average loss: 0.0048

In [7]:
torch.cuda.memory_allocated()

42101760

In [8]:
torch.cuda.memory_cached()

75497472

In [9]:
torch.cuda.empty_cache()

In [10]:
torch.cuda.memory_allocated()

42101760

In [11]:
torch.cuda.memory_cached()

54525952